In [4]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier

#######################################################################

path = 'C:\\Users\\yato\\PycharmProjects\\Selection\\house-votes-84.data'
names = ['Class Name',
         'handicapped-infants',
         'water-project-cost-sharing',
         'adoption-of-the-budget-resolution',
         'physician-fee-freeze',
         'el-salvador-aid',
         'religious-groups-in-schools',
         'anti-satellite-test-ban',
         'aid-to-nicaraguan-contras',
         'mx-missile',
         'immigration',
         'synfuels-corporation-cutback',
         'education-spending',
         'superfund-right-to-sue',
         'crime',
         'duty-free-exports',
         'export-administration-act-south-africa']

#######################################################################
#nettoyage et formatage
x = pd.read_csv(path, names=names)
X = x.drop(columns='Class Name')
drops = []
for i in range(len(X)):
    for value in X.values[i, :]:
        if value == '?':
            drops.append(i)
data = x.drop(index=drops)
X = data.drop(columns='Class Name')

y = data.values[:, 0]
for i in range(len(y)):
    if y[i] == 'democrat':
        y[i] = 1.0
    else:
        y[i] = 0.0

for i in range(len(X.values)):
    for j in range(len(X.values[i, :])):
        if X.values[i, j] == 'y':
            X.values[i, j] = 1.0
        else:
            X.values[i, j] = 0.0

            

#Penality Algorithm
cols = list(X.columns)
X_1 = X[cols]
X_1 = sm.add_constant(X_1)
model = LinearRegression().fit(X=X_1, y=y)
score = model.score(X=X_1, y=y)
print(score)

pmax = 1
while len(cols) > 0:
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y.astype(float),X_1.astype(float)).fit()
    p = pd.Series(model.pvalues.values[1:], index=cols)
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if pmax > 0.0000001:
        cols.remove(feature_with_p_max)
    else:
        break
print("Penality Algorithm")
print(cols)


X_1 = X[cols]
X_1 = sm.add_constant(X_1)
model = LinearRegression().fit(X=X_1, y=y)
score = model.score(X=X_1, y=y)
print(score)


print("RFE algorithm")
#RFE Algorithm
selected_features_BE = cols
print(selected_features_BE)

#number of features
nof_list=np.arange(1,16)
high_score=0
nof=0
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.6, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if score > high_score:
        high_score = score
        nof = nof_list[n]

print("Nombre optimal: %d" %nof)
print("score: %f" %  high_score)

#liste des attributs
cols = list(X.columns)
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model, nof)

#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)

#Fitting the data to model
model.fit(X_rfe,y)
temp = pd.Series(rfe.support_, index=cols)
selected_features_rfe = temp[temp is True]
print(selected_features_rfe)
print(selected_features_BE)





C:\Users\yato\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\yato\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


0.9042480646854771
Penality Algorithm
['physician-fee-freeze']
0.8843982291111642
RFE algorithm
['physician-fee-freeze']
Nombre optimal: 1
score: 0.910947
False
['physician-fee-freeze']
